In [ ]:
import pandas as pd
from lxml import etree
import re
from thefuzz import fuzz

In [ ]:
#ouverture du fichier XML grâce à etree

xml_file = '../../data/PO_t7/PO_t7.xml'
xml_tree = etree.parse(xml_file)

data = []

#on s'occupe des vedettes: pour l'instant, notre code ne prend pas en compte les vedettes avec + d'une valeur. A voir
#si il faut changer ça. sur le xpath cependant, on dirait qu'il n'y en a pas ?
for article in xml_tree.xpath('//article'):

#on enleve les localisationpa : inutiles pour nous si on ne se focalise que sur la france.
        if article.find('./localisationpa') is not None:
             continue
        vedette_i = article.xpath('./vedette/i')
        vedette_text = vedette_i[0].text if vedette_i else ''

#pareil: il y a un seul localisationde en double, mais il ne le prend pas en compte. A voir si il faut changer ça
        localisationde = article.find('./localisationde')
        localisationde_text = localisationde.text if localisationde is not None else 'none'

#idem. autre version du code qui en prend deux et sépare par ',' existe, à voir si c'est vraiment pertinent.
        localisationca = article.find('./localisationca')
        if localisationca is not None:
            localisationca_text = re.search(r'<localisationca>(.*?)<\/localisationca>', etree.tostring(localisationca, encoding=str)).group(1)
            '''
            cases:
                - 'c<sup>on</sup> de '
                - 'c<sup>on</sup> du '
                - 'c<sup>on</sup> d’'
                - 'c<sup>on</sup> de la '
                - 'c<sup>on</sup> de l’'
                - 'c<sup>on</sup> des '
            '''
            localisationca_text = re.sub(r"c<sup>on</sup> (de |d’|de la |du |de l’|des )", '', localisationca_text)
            
        else:
            localisationca_text = 'none'

#idem. Il n'y en a qu'un avec + d'une localisation -> l'autre étant en suisse. Utile ou pas? à voir.
        localisationco = article.find('./localisationco')
        if localisationco is not None:
            localisationco_text = re.search(r'<localisationco>(.*?)<\/localisationco>', etree.tostring(localisationco, encoding=str)).group(1)            
            '''
            cases:
                - 'c<sup>on</sup> de '
                - 'c<sup>on</sup> du '
                - 'c<sup>on</sup> d’'
                - 'c<sup>on</sup> de la '
                - 'c<sup>on</sup> de l’'
                - 'c<sup>on</sup> des '
                - 'c<sup>nes</sup> de '
                - 'c<sup>nes</sup> d’'
            '''
            localisationco_text = re.sub(r"c<sup>ne(s)?</sup> (de |d’|de la |du |de l’|des )", '', localisationco_text)
            
        else: 
            localisationco_text = 'none'
    
#on insère tout dans un dictionnaire        
        data.append({
            'old-id': article.get('old-id'),
            'vedette': vedette_text,
            'localisationde': localisationde_text,
            'localisationca': localisationca_text,
            'localisationco': localisationco_text
        })

In [ ]:
# tous les articles dans un df
df = pd.DataFrame(data)
df.head(25)

In [ ]:
# Ajout du dpt code au df
departements_df = pd.read_csv('departements-region.csv')

# On crée un dictionnaire avec les éléments qui nous intéressent de notre spreadsheet des départements
departements_dict = dict(zip(departements_df['dep_name'], departements_df['num_dep']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['num_dep'] = df['localisationde'].apply(lambda x: departements_dict.get(x, 'none'))

df.head(5)

In [ ]:
#on renomme pour plus de clarté.

df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune', 'num_dep' : 'dpt_code'})
df.head(5)

In [ ]:
#pour encore plus de clarté, nous réorganisons les colonnes 

df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco']]
df

## Liage des articles de type commune

In [ ]:
# Extraction des articles de type commune

commune_df = df[df['localisationco'] == 'none']

## Liage des communes de localisation (article dans une commune)

Il s’agit des lieux à l’échelle infra-communale. Ces lieux sont localisés dans une commune (`localisationco`)
…

In [ ]:
# Liage des articles localisés dans une commune

localisationco_df = df[df['localisationco'] != 'none']
localisationco_df['dpt_code'] = localisationco_df['dpt_code'].apply(lambda x: 'DEP_' + str(x) if x != 'none' else x)
localisationco_df.head(5)

In [ ]:
#On lie notre dataframe au référentiel insee avec de l'exact match.

main_insee_commune = pd.read_csv("main_insee_commune.tsv", delimiter='\t')

# on utilise merge pour le liage avec dpt_code et dep_id
merged_df = pd.merge(filtered_df, main_insee_commune, left_on='dpt_code', right_on='DEP_id', how='left')

# on sélectionne les lignes où la colonne 'localisationco' correspond à la colonne 'NCCENR' de main_insee_commune
# exact match
matching_rows = merged_df.loc[merged_df['localisationco'] == merged_df['NCCENR']]

# on crée une colonne 'insee_code' dans le dataframe filtered_df avec les codes correspondants à partir de main_insee_commune
filtered_df['insee_code'] = filtered_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values) > 0 else None, axis=1)
filtered_df['insee_code'] = filtered_df['insee_code'].fillna('none')


In [ ]:
filtered_df.head(5)

In [ ]:
# vérifier les communes de localisation liées sur exact match
# revoir commune

commune = filtered_df[filtered_df['insee_code'] != 'none']
commune.head(10)

#on a donc exactement 556 matches.

In [ ]:
#modification de notre dataframe: ajout d'une colonne method qui nous permet d'identifier la méthode utilisée pour le
#liage

commune['method'] = 'exact'
# marquer les liages exécutés sans possibilité de désambiguiser le dpt (pour validation manuelle a posteriori)
commune.loc[commune['dpt_code'] == 'none', 'method'] = 'nodpt'

print(commune)

In [ ]:
#les articles qui du coup n'ont pas de liage. on fera le fuzzy pour eux.

pascommune = filtered_df.loc[filtered_df['insee_code'] == 'none']
print(pascommune)

#nous en avons donc 124.

In [ ]:
# Définition d'une fonction pour le fuzzy join
## TODO insert NCCENR des candidats retournés par la fc fuzzy ?
## Mettre à jour cette fonction pour définir le ration Levenshtein appliqué ?

def fuzzy_join(row):
    # Filtre sur les lignes avec les mêmes départements
    main_dep = main_insee_commune[main_insee_commune['DEP_id'] == row['dpt_code']]
    # Calcul de la distance Levenshtein entre la vedette du pascommune et les NCCENR de chaque ligne de main_dep
    scores = main_dep['NCCENR'].apply(lambda x: fuzz.token_sort_ratio(x, row['vedette']))
    # Récupération des lignes avec une distance Levenshtein inférieure ou égale à 1
    filtered = main_dep[scores >= 90]
    # Ajout de la colonne insee_code dans le dataframe pascommune
    if not filtered.empty:
        insee_code = filtered['insee_code'].values[0]
        if pd.isnull(insee_code):
            return 'none'
        else:
            return insee_code
    else:
        return 'none'


# Application de la fonction sur chaque ligne du dataframe pascommune
pascommune['insee_code'] = pascommune.apply(fuzzy_join, axis=1)

In [ ]:
print(pascommune)

In [ ]:
#on veut savoir combien ont été joined par le fuzzy

fuzzy = pascommune[pascommune['insee_code'] != 'none']

# on ajoute une colonne method pour indiquer que le join a été réalisé par fuzzy

fuzzy['method'] = 'fuzzy'


print(fuzzy)

In [ ]:
# maintenant, on s'intéresse aux localisationco qui n'ont toujours pas de liage.

fuzzynull = pascommune[pascommune['insee_code'] == 'none']

#idem, on ajoute la méthode pour savoir que ces derniers n'ont été matchés ni avec le exact ni avec le fuzzy

fuzzynull['method'] = 'nulle'
print(fuzzynull)

In [ ]:
#il faut maintenant concaténer nos 3 dataframes, en respectant l'ordre original des article_id

merged_df = pd.concat([commune, fuzzy, fuzzynull], axis=0)
merged_df = merged_df.sort_values('method', ascending=True)

print(merged_df)

In [ ]:
merged_df['origine'] = 'avecloc'

In [ ]:
merged_df.to_csv("dataframe_avecloc_po7.csv", index=False)

In [ ]:
df

## Fusion et ordonnancement des liages